In [14]:
import urllib.robotparser
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver

In [15]:
al = urllib.robotparser.RobotFileParser()
al.set_url('https://www.booking.com/robots.txt')
al.read()
al.can_fetch('*','https://www.booking.com/hotel/vn/colline.vi.html?aid=397594;label=gog235jc-1DCAEoggI46AdIKlgDaPQBiAEBmAEquAEXyAEM2AED6AEB-AECiAIBqAIDuAKu4MX_BcACAdICJDNkOTdiNzhhLWU1MGQtNGNlMC04ZmQwLTRhYThkYmJjZGUwMdgCBOACAQ;sid=aa1ef10db21b3c99f2d46d4624c8ebf9;all_sr_blocks=408974501_130057956_0_2_0;checkin=2021-01-03;checkout=2021-01-04;dest_id=-3712045;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=408974501_130057956_0_2_0;hp_group_set=0;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=408974501_130057956_0_2_0__301643900;srepoch=1609658516;srpvid=023133c9b13f00e8;type=total;ucfs=1&#hotelTmpl')

True

In [111]:
next_page_url = 'https://www.booking.com/searchresults.vi.html?label=gen173nr-1DCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AED6AEBiAIBqAIDuAK9n8v_BcACAdICJGMxZjI0Y2VjLTI2YjMtNGFmNC04M2Y5LWE1YTQyMDA1NmE4ZtgCBOACAQ&lang=vi&sid=aa1ef10db21b3c99f2d46d4624c8ebf9&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.vi.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AED6AEBiAIBqAIDuAK9n8v_BcACAdICJGMxZjI0Y2VjLTI2YjMtNGFmNC04M2Y5LWE1YTQyMDA1NmE4ZtgCBOACAQ%3Bsid%3Daa1ef10db21b3c99f2d46d4624c8ebf9%3Bsb_price_type%3Dtotal%3Bsrpvid%3Df1b8780c7efe01c2%26%3B&ss=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&is_ski_area=0&ssne=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&ssne_untouched=TP.+H%C3%B4%CC%80+Chi%CC%81+Minh&dest_id=-3730078&dest_type=city&checkin_year=2021&checkin_month=1&checkin_monthday=5&checkout_year=2021&checkout_month=1&checkout_monthday=6&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1'
# while next_page_url != None:
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
browser.get(next_page_url)
time.sleep(10)
lpage_html = browser.page_source
lpage_tree = BeautifulSoup(lpage_html, 'html.parser')
l_hotels = lpage_tree.find_all('div', {'class': 'sr_property_block_main_row'})
browser.quit()
next_page_url = 'https://www.booking.com' + lpage_tree.find('a', {'title': 'Trang sau'})['href']

In [112]:
col = []
for i in range(len(l_hotels)):
    hotel_name = l_hotels[i].find('span', {'class': 'sr-hotel__name'}).text.strip()
    browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    hotel_url = 'https://www.booking.com' + l_hotels[i].find('a', {'class': 'js-sr-hotel-link hotel_name_link url'})['href'].strip()
    browser.get(hotel_url)
    time.sleep(10)
    hotel_html = browser.page_source
    hotel_tree = BeautifulSoup(hotel_html, 'html.parser')
    browser.quit()
    flag = 0
    first_room = hotel_tree.find('tr', {'class': 'js-rt-block-row hprt-table-cheapest-block hprt-table-cheapest-block-fix js-hprt-table-cheapest-block'})
    if first_room == None:
        flag = 1
    if flag == 0:
        room = hotel_tree.find_all(lambda tag: tag.name == 'tr' and tag.get('class') == ['js-rt-block-row'])
    else:
        first_room = hotel_tree.find('tr', {'class': 'js-rt-block-row hprt-table-cheapest-block hprt-table-cheapest-block-fix js-hprt-table-cheapest-block hprt-table-last-row'})
        if first_room == None:
            pass
        room = hotel_tree.find_all('tr', {'class': 'js-rt-block-row hprt-table-last-row'})
    x = first_room.find_all('span', {'class': 'hprt-facilities-facility'})
    y = first_room.find_all('div', {'class': 'hprt-facilities-facility'})
    for j in range(len(x)):
        if x[j].text.strip() not in col:
            col.append(x[j].text.strip())
        print(x[j].text.strip())
    for j in range(len(y)):
        if y[j].text.strip() not in col:
            col.append(y[j].text.strip())
        print(y[j].text.strip())
    price = first_room.find('div', {'class': 'hprt-price-block'}).find('div', {'class': 'bui-price-display__value prco-inline-block-maker-helper prco-font16-helper'}).text.strip()[4:]
    print(price)
    print('------')

    for j in range(len(room)):
        x = room[j].find_all('span', {'class': 'hprt-facilities-facility'})
        y = room[j].find_all('div', {'class': 'hprt-facilities-facility'})
        for k in range(len(x)):
            if x[k].text.strip() not in col:
                col.append(x[k].text.strip())
            print(x[k].text.strip())
        for k in range(len(y)):
            if y[k].text.strip() not in col:
                col.append(y[k].text.strip())
            print(y[k].text.strip())
        price = room[j].find('div', {'class': 'hprt-price-block'}).find('div', {'class': 'bui-price-display__value prco-inline-block-maker-helper prco-font16-helper'}).text.strip()[4:]
        print(price)
        print(f'---{i+1}---')
        

iêng
TV màn hình phẳng
Hệ thống cách âm
Phòng xông hơi
Minibar
3.696.000
---19---
4.395.006
---19---
Đồ vệ sinh cá nhân miễn phí
Áo choàng tắm
Két an toàn
Máy giặt
Dịch vụ streaming (như là Netflix)
Nhà vệ sinh
Bồn tắm hoặc Vòi sen
Khăn tắm
Ra trải giường
Ổ điện gần giường
Sản phẩm lau rửa
Sàn lát gạch/đá cẩm thạch
Bàn làm việc
TV
Điện thoại
Tiện nghi ủi
Truyền hình vệ tinh
Máy pha trà/cà phê
Bàn ủi
Đài radio
Có phòng thông nhau qua cửa nối
Máy sấy tóc
Đầu đĩa DVD
Khăn tắm/Bộ khăn trải giường (có thu phí)
Dịch vụ báo thức
Sàn trải thảm
Ấm đun nước điện
Truyền hình cáp
Dịch vụ báo thức
Đồng hồ báo thức
Tủ hoặc phòng để quần áo
Các tầng trên đi lên bằng thang máy
Giá treo quần áo
Giấy vệ sinh
Nắp che ổ cắm điện an toàn
30 m²
Nhìn ra thành phố
Hồ bơi có tầm nhìn
Hồ bơi trên sân thượng
Điều hòa không khí
Phòng tắm riêng
TV màn hình phẳng
Hệ thống cách âm
Phòng xông hơi
Minibar
3.696.000
---19---
4.395.006
---19---
Đồ vệ sinh cá nhân miễn phí
Áo choàng tắ

In [107]:
temp_col = col

['Đồ vệ sinh cá nhân miễn phí',
 'Vòi sen',
 'Áo choàng tắm',
 'Két an toàn',
 'Nhà vệ sinh',
 'Khăn tắm',
 'Bàn làm việc',
 'Khu vực tiếp khách',
 'TV',
 'Dép',
 'Tủ lạnh',
 'Điện thoại',
 'Hệ thống sưởi',
 'Máy sấy tóc',
 'Sàn trải thảm',
 'Ấm đun nước điện',
 'Truyền hình cáp',
 'Dịch vụ báo thức',
 'Tủ hoặc phòng để quần áo',
 'Giá treo quần áo',
 '29 m²',
 'Nhìn ra thành phố',
 'Bồn tắm',
 'Điều hòa không khí',
 'Phòng tắm riêng',
 'TV màn hình phẳng',
 'Hệ thống cách âm',
 'Minibar',
 'WiFi miễn phí',
 'Ghế sofa',
 'Máy fax',
 'Phòng thay quần áo',
 '56 m²',
 '61 m²',
 'Nhìn ra hồ bơi',
 '87 m²',
 'Ổ điện gần giường',
 'Toilet phụ',
 'Dịch vụ báo thức',
 'Đồng hồ báo thức',
 'Giấy vệ sinh',
 '18 m²',
 'Phòng tắm riêng trong phòng',
 '24 m²',
 'Bồn tắm hoặc Vòi sen',
 'Tiện nghi ủi',
 'Giường cực dài (> 2 mét)',
 '36 m²',
 '42 m²',
 '50 m²',
 'Bếp',
 'Bếp nhỏ',
 'Khu vực phòng ăn',
 'Bàn ăn',
 '100 m²',
 'Sàn lát gỗ',
 '20 m²',
 'Hồ b

In [108]:
len(temp_col)

131

In [99]:
browser_temp = webdriver.Chrome(executable_path='./chromedriver.exe')
# url_temp = 'https://www.booking.com/hotel/vn/tan-son-nhat-saigon.vi.html?label=gen173nr-1DCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AED6AEBiAIBqAIDuAK9n8v_BcACAdICJGMxZjI0Y2VjLTI2YjMtNGFmNC04M2Y5LWE1YTQyMDA1NmE4ZtgCBOACAQ;sid=aa1ef10db21b3c99f2d46d4624c8ebf9;all_sr_blocks=49942705_86257607_0_1_0;checkin=2021-01-04;checkout=2021-01-05;dest_id=-3730078;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=2;highlighted_blocks=49942705_86257607_0_1_0;hpos=2;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=49942705_86257607_0_1_0__240500000;srepoch=1609767241;srpvid=20ab5f64fd750074;type=total;ucfs=1&#hotelTmpl'
url_temp = 'https://www.booking.com/hotel/vn/aem-corner-saigon.vi.html?label=gen173nr-1DCAEoggI46AdIM1gEaPQBiAEBmAEquAEXyAEM2AED6AEBiAIBqAIDuAK9n8v_BcACAdICJGMxZjI0Y2VjLTI2YjMtNGFmNC04M2Y5LWE1YTQyMDA1NmE4ZtgCBOACAQ;sid=aa1ef10db21b3c99f2d46d4624c8ebf9;all_sr_blocks=25182329_86455561_0_2_0;checkin=2021-01-04;checkout=2021-01-05;dest_id=-3730078;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=31;highlighted_blocks=25182329_86455561_0_2_0;hpos=6;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=25182329_86455561_0_2_0__65000000;srepoch=1609768176;srpvid=f5ea6137ce8d0126;type=total;ucfs=1&#hotelTmpl'
browser_temp.get(url_temp)
time.sleep(10)
temp_html = browser_temp.page_source
temp_tree = BeautifulSoup(temp_html, 'html.parser')
browser_temp.quit()

In [103]:
# price = temp_tree.find_all('td', {'class': 'd_pd_hp_price_left_align hprt-table-cell hprt-table-cell-price'})
# temp_facilities = temp_tree.find_all('td', {'class': 'hprt-table-cell -first hprt-table-cell-roomtype droom_seperator'})
# print(len(price))
# print(len(temp_facilities))
flag = 0
first_room = temp_tree.find('tr', {'class': 'js-rt-block-row hprt-table-cheapest-block hprt-table-cheapest-block-fix js-hprt-table-cheapest-block'})

if first_room == None:
    flag = 1
if flag == 0:
    room = temp_tree.find_all(lambda tag: tag.name == 'tr' and tag.get('class') == ['js-rt-block-row'])
else:
    first_room = temp_tree.find('tr', {'class': 'js-rt-block-row hprt-table-cheapest-block hprt-table-cheapest-block-fix js-hprt-table-cheapest-block hprt-table-last-row'})
    room = temp_tree.find_all('tr', {'class': 'js-rt-block-row hprt-table-last-row'})
x = first_room.find_all('span', {'class': 'hprt-facilities-facility'})
y = first_room.find_all('div', {'class': 'hprt-facilities-facility'})
for i in range(len(x)):
    print(x[i].text.strip())
for i in range(len(y)):
    print(y[i].text.strip())
f_price = first_room.find('div', {'class': 'hprt-price-block'}).find('div', {'class': 'bui-price-display__value prco-inline-block-maker-helper prco-font16-helper'}).text.strip()[4:]
print(f_price)
print('------')

for j in range(len(room)):
    x = room[j].find_all('span', {'class': 'hprt-facilities-facility'})
    y = room[j].find_all('div', {'class': 'hprt-facilities-facility'})
    for i in range(len(x)):
        print(x[i].text.strip())
    for i in range(len(y)):
        print(y[i].text.strip())
    f_price = room[j].find('div', {'class': 'hprt-price-block'}).find('div', {'class': 'bui-price-display__value prco-inline-block-maker-helper prco-font16-helper'}).text.strip()[4:]
    print(f_price)
    print('------')

Đồ vệ sinh cá nhân miễn phí
Áo choàng tắm
Két an toàn
Nhà vệ sinh
Bồn tắm hoặc Vòi sen
Sàn lát gỗ
Khăn tắm
Ra trải giường
Ổ điện gần giường
Bàn làm việc
TV
Dép
Điện thoại
Truyền hình vệ tinh
Máy pha trà/cà phê
Máy sấy tóc
Quạt máy
Sàn trải thảm
Ấm đun nước điện
Truyền hình cáp
Dịch vụ báo thức
Tủ hoặc phòng để quần áo
Giá treo quần áo
Giấy vệ sinh
20 m²
Nhìn ra thành phố
Hồ bơi có tầm nhìn
Hồ bơi trên sân thượng
Điều hòa không khí
Phòng tắm riêng trong phòng
TV màn hình phẳng
Hệ thống cách âm
Minibar
WiFi miễn phí
650.000
------
Đồ vệ sinh cá nhân miễn phí
Áo choàng tắm
Két an toàn
Nhà vệ sinh
Bồn tắm hoặc Vòi sen
Sàn lát gỗ
Khăn tắm
Ra trải giường
Ổ điện gần giường
Bàn làm việc
TV
Dép
Điện thoại
Truyền hình vệ tinh
Máy pha trà/cà phê
Máy sấy tóc
Quạt máy
Sàn trải thảm
Ấm đun nước điện
Truyền hình cáp
Dịch vụ báo thức
Tủ hoặc phòng để quần áo
Giá treo quần áo
Giấy vệ sinh
20 m²
Nhìn ra thành phố
Hồ bơi có tầm nhìn
Hồ bơi trên sân thượ